In [1]:
%run gpt_model.ipynb


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/Users/lemi/gpt/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/lemi/gpt/.venv/lib/python3.12/site-packages/traitlets/config/application.py", l

In [10]:
import os

In [2]:
with open('wizard.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))

with open("vocab.txt", "w", encoding="utf-8") as f:
    for ch in chars:
        f.write(ch + "\n")
        
vocab_size = len(chars)

In [3]:
char_to_int = {ch:i for i,ch in enumerate(chars)}
encode = lambda s: [char_to_int[c] for c in s]

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [13]:
if os.path.exists("model"):
    m = torch.load("model")
    print("model loaded")
else:
    model = GPTLanguageModel(vocab_size)
    m = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# model.parameters() -> all trainable weights in the model
# Each parameter tensor has .data and .grad

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss {losses['train']:.3f}, val loss {losses['val']:.3f}")
    
    xb, yb = get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # parameter.grad is None
    loss.backward()
    optimizer.step()
print(loss.item())

torch.save(m, "model")
print("model saved")

model loaded
step: 0, train loss 2.050, val loss 2.145
step: 250, train loss 2.046, val loss 2.147
step: 500, train loss 2.037, val loss 2.128
step: 750, train loss 2.045, val loss 2.124
2.3633997440338135
model saved
